In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 7_compare_anisotropy_profiles.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Compare the anisotropy profiles of the remnant separated by their constant 
anisotropy.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, dill as pickle, logging, time

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu
from astropy import constants as apc

## Analysis
import scipy.stats
import scipy.interpolate

## galpy
from galpy import orbit
from galpy import potential

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import fitting as pfit
from tng_dfs import kinematics as pkin
from tng_dfs import plot as pplot
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/5_compare_distribution_functions/7_compare_anisotropies/')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Compute the anisotropies

In [ ]:
verbose = True

# Pathing
dens_fitting_dir = os.path.join(fitting_dir_base,'density_profile')
df_fitting_dir = os.path.join(fitting_dir_base,'distribution_function')
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Anisitropy calculation parameters
n_bs = 10
compute_betas_kwargs = {'use_dispersions':True,
                        'return_kinematics':False}

# # Get the sample orbits
# sample_data_cb = np.load(os.path.join(analysis_dir,'sample_data_cb.npy'),
#     allow_pickle=True)
# sample_data_om = np.load(os.path.join(analysis_dir,'sample_data_om.npy'),
#     allow_pickle=True)
# sample_data_om2 = np.load(os.path.join(analysis_dir,'sample_data_om2.npy'),
#     allow_pickle=True)

# Begin logging
os.makedirs('./log/',exist_ok=True)
log_filename = './log/7_compare_anisotropies.log'
if os.path.exists(log_filename):
    os.remove(log_filename)
logging.basicConfig(filename=log_filename, level=logging.INFO, filemode='w', 
    force=True)
logging.info('Beginning anisotropy calculation. Time: '+\
             time.strftime('%a, %d %b %Y %H:%M:%S',time.localtime()))

anisotropy_data_list = []

for i in range(n_mw):
    # if i != 0: continue

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    pid = co.get_property('stars','ParticleIDs')

    for j in range(n_major):
        # if j > 0: continue

        if verbose: 
            msg = f'Calculating anisotropy profiles for MW analog '+\
                  f'{i+1}/{n_mw}, merger {j+1}/{n_major}'
            logging.info(msg)
            print(msg, end='\r')

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        upid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        indx = np.where(np.isin(pid,upid))[0]
        orbs = co.get_orbs('stars')[indx]
        n_star = len(orbs)
        star_mass = co.get_masses('stars')[indx].to_value(apu.Msun)
        r = orbs.r().to_value(apu.kpc)
        r_softening = putil.get_softening_length('stars', z=0, physical=True)
        rmin = np.max([np.min(r), r_softening])

        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
        )

        ### Compute anisotropy using the standard binning

        # Define the adaptive binning keyword dict and then bin
        n_bin = np.min([500, n_star//10]) # n per bin
        adaptive_binning_kwargs = {'n':n_bin,
                                   'rmin':rmin,
                                   'rmax':np.max(r),
                                   'bin_mode':'exact numbers',
                                   'bin_equal_n':True,
                                   'end_mode':'ignore',
                                   'bin_cents_mode':'median'}
        bin_edges, bin_cents, _ = pkin.get_radius_binning(orbs, 
            **adaptive_binning_kwargs)
        bin_size = bin_edges[1:] - bin_edges[:-1]

        # Compute the anisotropy profile
        betas = pkin.compute_betas_bootstrap(orbs, bin_edges, 
            n_bootstrap=n_bs, compute_betas_kwargs=compute_betas_kwargs)
        _data += (bin_edges, bin_cents, betas)

        ### Compute anisotropy using a fixed-number binning scheme

        # Define the fixed binning keyword dict and then bin
        num_bins_fixed = 50
        n_bin = n_star//num_bins_fixed # n per bin
        fixed_binning_kwargs = {'n':n_bin,
                                'rmin':rmin,
                                'rmax':np.max(r),
                                'bin_mode':'exact numbers',
                                'bin_equal_n':True,
                                'end_mode':'ignore',
                                'bin_cents_mode':'median'}
        bin_edges_fixed, bin_cents_fixed, _ = pkin.get_radius_binning(orbs,
            **fixed_binning_kwargs)
        
        # Compute the anisotropy profile
        betas_fixed = pkin.compute_betas_bootstrap(orbs, bin_edges_fixed, 
            n_bootstrap=n_bs, compute_betas_kwargs=compute_betas_kwargs)
        _data += (num_bins_fixed, bin_edges_fixed, bin_cents_fixed, betas_fixed)

        anisotropy_data_list.append(_data)
        

# Save as a structured array
anisotropy_data_dtype = np.dtype([
    ('z0_sid',int),
    ('major_acc_sid',int),
    ('major_mlpid',int),
    ('merger_number',int),
    ('bin_edges',object),
    ('bin_cents',object),
    ('betas',object),
    ('num_bins_fixed',int),
    ('bin_edges_fixed',object),
    ('bin_cents_fixed',object),
    ('betas_fixed',object),
])
anisotropy_data = np.array(anisotropy_data_list, dtype=anisotropy_data_dtype)
np.save(os.path.join(analysis_dir, 'anisotropy_data.npy'), 
        anisotropy_data)

### Get the stashed data

In [ ]:
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Or load the structured arrays
# moms = [1,2,3,4]
anisotropy_data_filename = os.path.join(analysis_dir, 'anisotropy_data.npy')
anisotropy_data = np.load(anisotropy_data_filename, allow_pickle=True)

# Load the merger information
merger_data = np.load(os.path.join(analysis_dir,'merger_data.npy'), 
    allow_pickle=True)

checks = True
if checks:
    assert np.all( anisotropy_data['z0_sid'] == merger_data['z0_sid'] ), \
        'Something went wrong'
    assert np.all( anisotropy_data['major_acc_sid'] == merger_data['major_acc_sid'] ), \
        'Something went wrong'
    assert np.all( anisotropy_data['major_mlpid'] == merger_data['major_mlpid'] ), \
        'Something went wrong'

### Plot all of the anisotropy profiles, separeted by the constant anisotropy value

In [ ]:
columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()

fig = plt.figure(figsize=(textwidth,3))
axs = fig.subplots(nrows=1, ncols=3)

anisotropy_cuts = [0.3,0.7]
anisotropy_ranges = [[-9999,anisotropy_cuts[0]],
                     [anisotropy_cuts[0],anisotropy_cuts[1]],
                     [anisotropy_cuts[1],9999]]

cba = merger_data['anisotropy']
scale_radius = merger_data['a']

colors = ['Red', 'ForestGreen', 'DodgerBlue']

for i in range(3):

    indx = np.where( (cba > anisotropy_ranges[i][0]) & 
                     (cba < anisotropy_ranges[i][1]) )[0]
    interps = []
    
    # Scale alpha inversely to the number of mergers in indx for visibility
    alpha = 0.5*(1-(np.log10(len(indx))/np.log10(len(cba)))**0.8)
    # print(alpha)

    for j in range(len(indx)):
        betas = np.median( anisotropy_data['betas'][indx[j]], axis=0 )
        bin_cents = anisotropy_data['bin_cents'][indx[j]]
        
        axs[i].plot(bin_cents, betas, # label=f'{indx[j]}')
                    color=colors[i], alpha=alpha)
        
        interp = scipy.interpolate.interp1d(bin_cents, betas, 
            kind='linear', fill_value=np.nan, bounds_error=False)
        interps.append(interp)
    
    bin_cents_plot = np.logspace(-0.5,2,100)
    betas_plot = np.array([interp(bin_cents_plot) for interp in interps])
    betas_plot = np.nanmedian(betas_plot, axis=0)
    axs[i].plot(bin_cents_plot, betas_plot, 
                color=colors[i], linewidth=2)

    axs[i].set_xlabel(r'$r$ [kpc]')
    axs[i].set_xscale('log')
    axs[i].set_ylim(-0.2,1)

    if i > 0:
        axs[i].tick_params(labelleft=False)
    else:
        axs[i].set_ylabel(r'$\beta$')

# Label the spaces
text = [r'$\beta < '+str(anisotropy_cuts[0])+r'$', 
        r'$'+str(anisotropy_cuts[0])+r' < \beta < '+str(anisotropy_cuts[1])+r'$', 
        r'$\beta > '+str(anisotropy_cuts[1])+r'$']
for i in range(3):
    axs[i].text(0.05, 0.95, text[i], transform=axs[i].transAxes, 
                fontsize=12, verticalalignment='top')

fig.tight_layout()
fig.subplots_adjust(wspace=0.05)
fig.savefig(os.path.join(local_fig_dir,
                         'anisotropy_profile_comparison.pdf'),
            dpi=300, bbox_inches='tight')
fig.show()

### Same plot but with fixed binning

In [ ]:
columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()

fig = plt.figure(figsize=(textwidth,3))
axs = fig.subplots(nrows=1, ncols=3)

anisotropy_cuts = [0.3,0.7]
anisotropy_ranges = [[-9999,anisotropy_cuts[0]],
                     [anisotropy_cuts[0],anisotropy_cuts[1]],
                     [anisotropy_cuts[1],9999]]

cba = merger_data['anisotropy']
scale_radius = merger_data['a']

colors = ['Red', 'ForestGreen', 'DodgerBlue']

for i in range(3):

    indx = np.where( (cba > anisotropy_ranges[i][0]) & 
                     (cba < anisotropy_ranges[i][1]) )[0]
    interps = []
    
    # Scale alpha inversely to the number of mergers in indx for visibility
    alpha = 0.5*(1-(np.log10(len(indx))/np.log10(len(cba)))**0.8)
    # print(alpha)

    for j in range(len(indx)):
        betas = np.median( anisotropy_data['betas_fixed'][indx[j]], axis=0 )
        bin_cents = anisotropy_data['bin_cents_fixed'][indx[j]]
        
        axs[i].plot(bin_cents, betas, # label=f'{indx[j]}')
                    color=colors[i], alpha=alpha)
        
        interp = scipy.interpolate.interp1d(bin_cents, betas, 
            kind='linear', fill_value=np.nan, bounds_error=False)
        interps.append(interp)
    
    bin_cents_plot = np.logspace(-0.5,2,100)
    betas_plot = np.array([interp(bin_cents_plot) for interp in interps])
    betas_plot = np.nanmedian(betas_plot, axis=0)
    axs[i].plot(bin_cents_plot, betas_plot, 
                color=colors[i], linewidth=2)

    axs[i].set_xlabel(r'$r$ [kpc]')
    axs[i].set_xscale('log')
    axs[i].set_ylim(-0.2,1)

    if i > 0:
        axs[i].tick_params(labelleft=False)
    else:
        axs[i].set_ylabel(r'$\beta$')

# Label the spaces
text = [r'$\beta < '+str(anisotropy_cuts[0])+r'$', 
        r'$'+str(anisotropy_cuts[0])+r' < \beta < '+str(anisotropy_cuts[1])+r'$', 
        r'$\beta > '+str(anisotropy_cuts[1])+r'$']
for i in range(3):
    axs[i].text(0.05, 0.95, text[i], transform=axs[i].transAxes, 
                fontsize=12, verticalalignment='top')

fig.tight_layout()
fig.subplots_adjust(wspace=0.05)
fig.savefig(os.path.join(local_fig_dir,
                         'anisotropy_profile_comparison_fixed_bins.png'),
            dpi=300, bbox_inches='tight')
fig.show()

### Same plot but scaled by the density profile scale radius

In [ ]:
columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()

fig = plt.figure(figsize=(textwidth,3))
axs = fig.subplots(nrows=1, ncols=3)

anisotropy_cuts = [0.3,0.7]
anisotropy_ranges = [[-9999,anisotropy_cuts[0]],
                     [anisotropy_cuts[0],anisotropy_cuts[1]],
                     [anisotropy_cuts[1],9999]]

cba = merger_data['anisotropy']
scale_radius = merger_data['a']
om_scale_radius = merger_data['ra']

colors = ['Red', 'ForestGreen', 'DodgerBlue']

for i in range(3):

    indx = np.where( (cba > anisotropy_ranges[i][0]) & 
                     (cba < anisotropy_ranges[i][1]) )[0]
    interps = []
    
    # Scale alpha inversely to the number of mergers in indx for visibility
    alpha = 0.5*(1-(np.log10(len(indx))/np.log10(len(cba)))**0.8)
    # print(alpha)

    for j in range(len(indx)):
        betas = np.median( anisotropy_data['betas_fixed'][indx[j]], axis=0 )
        bin_cents = anisotropy_data['bin_cents_fixed'][indx[j]]/scale_radius[indx[j]]
        
        axs[i].plot(bin_cents, betas, # label=f'{indx[j]}')
                    color=colors[i], alpha=alpha)
        
        interp = scipy.interpolate.interp1d(bin_cents, betas, 
            kind='linear', fill_value=np.nan, bounds_error=False)
        interps.append(interp)
    
    bin_cents_plot = np.logspace(-2,2,100)
    betas_plot = np.array([interp(bin_cents_plot) for interp in interps])
    betas_plot = np.nanmedian(betas_plot, axis=0)
    axs[i].plot(bin_cents_plot, betas_plot, 
                color=colors[i], linewidth=2)

    axs[i].set_xlabel(r'$r/a$')
    axs[i].set_xscale('log')
    axs[i].set_ylim(-0.2,1)

    if i > 0:
        axs[i].tick_params(labelleft=False)
    else:
        axs[i].set_ylabel(r'$\beta$')

# Label the spaces
text = [r'$\beta < '+str(anisotropy_cuts[0])+r'$', 
        r'$'+str(anisotropy_cuts[0])+r' < \beta < '+str(anisotropy_cuts[1])+r'$', 
        r'$\beta > '+str(anisotropy_cuts[1])+r'$']
for i in range(3):
    axs[i].text(0.05, 0.95, text[i], transform=axs[i].transAxes, 
                fontsize=12, verticalalignment='top')

fig.tight_layout()
fig.subplots_adjust(wspace=0.05)
fig.savefig(os.path.join(local_fig_dir,
                         'anisotropy_profile_comparison_scaled_density.png'),
            dpi=300, bbox_inches='tight')
fig.show()

In [ ]:
columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()

fig = plt.figure(figsize=(textwidth,3))
axs = fig.subplots(nrows=1, ncols=3)

anisotropy_cuts = [0.3,0.7]
anisotropy_ranges = [[-9999,anisotropy_cuts[0]],
                     [anisotropy_cuts[0],anisotropy_cuts[1]],
                     [anisotropy_cuts[1],9999]]

cba = merger_data['anisotropy']
scale_radius = merger_data['a']
om_scale_radius = merger_data['ra']

colors = ['Red', 'ForestGreen', 'DodgerBlue']

for i in range(3):

    indx = np.where( (cba > anisotropy_ranges[i][0]) & 
                     (cba < anisotropy_ranges[i][1]) )[0]
    interps = []
    
    # Scale alpha inversely to the number of mergers in indx for visibility
    alpha = 0.5*(1-(np.log10(len(indx))/np.log10(len(cba)))**0.7)
    # print(alpha)

    for j in range(len(indx)):
        betas = np.median( anisotropy_data['betas_fixed'][indx[j]], axis=0 )
        bin_cents = anisotropy_data['bin_cents_fixed'][indx[j]]/om_scale_radius[indx[j]]
        
        axs[i].plot(bin_cents, betas, # label=f'{indx[j]}')
                    color=colors[i], alpha=alpha)
        
        interp = scipy.interpolate.interp1d(bin_cents, betas, 
            kind='linear', fill_value=np.nan, bounds_error=False)
        interps.append(interp)
    
    bin_cents_plot = np.logspace(-2,2,100)
    betas_plot = np.array([interp(bin_cents_plot) for interp in interps])
    betas_plot = np.nanmedian(betas_plot, axis=0)
    axs[i].plot(bin_cents_plot, betas_plot, 
                color=colors[i], linewidth=2)

    axs[i].set_xlabel(r'$r/r_{a}$')
    axs[i].set_xscale('log')
    axs[i].set_ylim(-0.2,1)

    if i > 0:
        axs[i].tick_params(labelleft=False)
    else:
        axs[i].set_ylabel(r'$\beta$')

# Label the spaces
text = [r'$\beta < '+str(anisotropy_cuts[0])+r'$', 
        r'$'+str(anisotropy_cuts[0])+r' < \beta < '+str(anisotropy_cuts[1])+r'$', 
        r'$\beta > '+str(anisotropy_cuts[1])+r'$']
for i in range(3):
    axs[i].text(0.05, 0.95, text[i], transform=axs[i].transAxes, 
                fontsize=12, verticalalignment='top')

fig.tight_layout()
fig.subplots_adjust(wspace=0.05)
fig.savefig(os.path.join(local_fig_dir,
                         'anisotropy_profile_comparison_scaled_om.png'),
            dpi=300, bbox_inches='tight')
fig.show()